# 1. Data Preproccesing
Data quantitiy : 500

In [ ]:
import os
# file_titles = os.listdir("../fluffyping-imgs (jinho)")
file_titles = os.listdir("./img/fluffyping-imgs")

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torch.autograd import Variable
import torch.optim as optim

import matplotlib.pyplot as plt

In [2]:
#batch_size
batch_size = 16
learning_rate = 0.002
num_epoch = 200

In [4]:
class myCNN(nn.Module):
    def __init__ (self):
        super(myCNN, self).__init__()
        # 1단계 CNN Layer
        # img = 32 x 32 x 3
        self.cnn_layer = nn.Sequential(
            # Conv Layer + relu
            nn.Conv2d(3, 16, 5, padding=2), # 32 x 32 x 16
            nn.ReLU(),

            # Conv Layer + relu
            nn.Conv2d(16, 32, 5, padding=2), # 32 x 32 x 32
            nn.ReLU(),

            # pool 32 x 32 -> 16 x 16
            nn.MaxPool2d(2,2),

            # Conv Layer + relu
            nn.Conv2d(32, 64, 5, padding=2), # 16 x 16 x 64
            nn.ReLU(),

            # pool : 16 x 16 -> 8 x 8
            nn.MaxPool2d(2,2)
        ) # 8 x 8 x 64
        # 2단계 FC Layer
        self.fc_layer = nn.Sequential(
            nn.Linear(8*8*64, 100),
            nn.ReLU(),
            nn.Linear(100, 3)
        )

    def forward(self, x):
        out = self.cnn_layer(x) # out ; batch_size x 8 x 8 x 64 3d tensor
        out = out.view(batch_size, -1) # out : batch_size x 8*8*64 2d tensor
        out = self.fc_layer(out) # fc layer input : 8*8*64*1 1d tensor
        return out

In [5]:
# train dataset folder
full_dataset = dset.ImageFolder("./dataset", transform=transforms.Compose([ 
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor()
]))

In [11]:
splitting = [int(len(full_dataset)*0.6), int(len(full_dataset)*0.2), int(len(full_dataset)*0.2)]
if sum(splitting) != len(full_dataset):
    splitting[0] += len(full_dataset) - sum(splitting)
print(sum(splitting), len(full_dataset))
train_dataset, test_dataset, val_dataset = random_split(full_dataset, splitting)

913 913


In [12]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    )

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    )

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = myCNN().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

cuda


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [9]:
def EstimateAccuracy(dloader, imodel):
    correct = 0
    total = 0

    for image, label in dloader:
        x = Variable(image, volatile=True).to(device)
        y = Variable(label).to(device)

        y_hat = imodel.forward(x)
        _, y_hat_index = torch.max(y_hat, 1)

        total += label.size(0)
        correct += (y_hat_index == y).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))
    return 100 * correct / total

In [10]:
loss_arr = []
accu_arr = []

for i in range(num_epoch):
    for image, label in train_loader:
        x = Variable(image).to(device)
        y = Variable(label).to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print("%d.." % i)
        loss_arr.append(loss)
        accu_arr.append(EstimateAccuracy(val_loader, model))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader

# # CNN 모델 정의
# class SimpleCNN(nn.Module):
#     def __init__(self):
#         super(SimpleCNN, self).__init__()
#         self.conv_layers = nn.Sequential(
#             # 첫 번째 Convolution Block
#             nn.Conv2d(1, 32, kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2),
            
#             # 두 번째 Convolution Block
#             nn.Conv2d(32, 64, kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2)
#         )
        
#         self.fc_layers = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(64 * 5 * 5, 128),
#             nn.ReLU(),
#             nn.Linear(128, 10)
#         )
        
#     def forward(self, x):
#         x = self.conv_layers(x)
#         x = self.fc_layers(x)
#         return x

# # 학습 함수 정의
# def train(model, train_loader, criterion, optimizer, device):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
        
#         optimizer.zero_grad()
#         output = model(data)
#         loss = criterion(output, target)
#         loss.backward()
#         optimizer.step()
        
#         if batch_idx % 100 == 0:
#             print(f'Train Batch: {batch_idx}/{len(train_loader)} '
#                   f'Loss: {loss.item():.4f}')

# # 테스트 함수 정의
# def test(model, test_loader, criterion, device):
#     model.eval()
#     test_loss = 0
#     correct = 0
    
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += criterion(output, target).item()
#             pred = output.argmax(dim=1, keepdim=True)
#             correct += pred.eq(target.view_as(pred)).sum().item()
    
#     test_loss /= len(test_loader)
#     accuracy = 100. * correct / len(test_loader.dataset)
#     print(f'Test set: Average loss: {test_loss:.4f}, '
#           f'Accuracy: {correct}/{len(test_loader.dataset)} '
#           f'({accuracy:.2f}%)')

    
# if __name__ == '__main__':
#     # 장치 설정
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
#     # 데이터 전처리
#     transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
#     ])
    
#     # MNIST 데이터셋 로드
#     train_dataset = torchvision.datasets.MNIST(
#         root='./data', 
#         train=True, 
#         download=True, 
#         transform=transform
#     )
#     test_dataset = torchvision.datasets.MNIST(
#         root='./data', 
#         train=False, 
#         transform=transform
#     )
    
#     # 데이터 로더 생성
#     train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#     test_loader = DataLoader(test_dataset, batch_size=1000)
    
#     # 모델 초기화
#     model = SimpleCNN().to(device)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.001)
    
#     # 학습 실행
#     epochs = 5
#     for epoch in range(1, epochs + 1):
#         print(f'\nEpoch: {epoch}')
#         train(model, train_loader, criterion, optimizer, device)
#         test(model, test_loader, criterion, device)


Failed to download (trying next):
HTTP Error 403: Forbidden



 96%|█████████▌| 9535488/9912422 [00:15<00:00, 602575.96it/s]


KeyboardInterrupt: 

In [ ]:
images, labels = next(iter(train_loader))
print(images[0], labels[0])

tensor([[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242,  0.3013,  1.7396,  2